In [1]:
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import os
import time
import copy

import tensorflow as tf
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

import os 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import geopandas
import geoplot
import pandas as pd
import geoplot as gplt
import geoplot.crs as gcrs
import geopandas as gpd

from shapely import wkt

from PIL import Image

ModuleNotFoundError: No module named 'torch'

In [ ]:
DATA_DIR = '../../data/'
MODEL_DIR = os.path.join(DATA_DIR, 'model')
GRAPH_DIR = os.path.join(DATA_DIR, 'graphs')

In [ ]:
def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB') 

In [ ]:
def get_input_transform():   
    transf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])    

    return transf

In [ ]:
def get_input_tensors(img):
    transf = get_input_transform()
    return transf(img).unsqueeze(0)

In [ ]:
def prediction_image_path(image_path, model):
    """ Sort les predictions d'un model sur l'image
        passee en arguement.
    """
    img = get_image(image_path)
    
    model = model.to(device)
    model.eval()
    
    img_t = get_input_tensors(img)
    img_t = img_t.to(device)
    
    output = model(img_t)
    _, pred = torch.max(output, 1)
    
    pred = pred.detach().cpu().numpy()[0]
    
    return pred

In [ ]:
def run_model(description):
    
    #set up le GPU
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print('%d GPU(s) disponible.' % torch.cuda.device_count())
        print('GPU utilisé :', torch.cuda.get_device_name(0))

    else:
        print('Pas de GPU disponible, utilisation du CPU.')
        device = torch.device("cpu")
    
    #charge le dataset
    
    data_preds = pd.read_csv(os.path.join(DATA_DIR, 'images_label_' + description + '.csv')
    
    #charge le modele
    
    model_euroSAT = models.resnet50(pretrained=False, progress=True)
    num_ftrs = model_euroSAT.fc.in_features
    model_euroSAT.fc = nn.Linear(num_ftrs, 2)
    model_euroSAT.load_state_dict(torch.load(os.path.join(MODEL_DIR, )))
    
    #run le modele
    
    pred_euroSAT = []

    for idx, obs in damas_preds.iterrows():
        image_nom = obs['nom_images_torchvision']
        image_path = os.path.join('/content/', 'damas', obs['segment'], str(obs['label']), image_nom)
        #pred_no_euroSAT.append(prediction_image_path(image_path, model_no_euroSAT))
        pred_euroSAT.append(prediction_image_path(image_path, model_euroSAT))
        
    data_preds['pred_euroSAT'] = pred_euroSAT
    
    #sauvegarde le modèle
    
    damas_preds.to_csv(os.path.join(DATA_DIR, 'images_label&pred_' + description +'.csv', index=False)
    
    return

In [ ]:
def make_map():
    
    #charge le data set
    data = pd.read_csv(#chercher le csv ecrit par run_model)
    
    #prepare les données
    data['geometry'] = data['centroide']
    data['Coordinates'] = data['centroide'].apply(wkt.loads)
    data = data.set_geometry('Coordinates')
    
        
    #fait la map
    
    fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(20, 10), subplot_kw={'projection': gcrs.WebMercator()})

    gplt.webmap(data, projection=gcrs.WebMercator(), ax=axes[0])
    gplt.pointplot(data[data['label'] == 1], ax=axes[0], alpha=1, s=2, color='k')
    gplt.kdeplot(data_preds[data['pred_euroSAT'] == 1], cmap='RdYlGn_r', n_levels=30,
                 shade=True, shade_lowest=False, ax=axes[0], alpha=0.5)

    gplt.webmap(data, projection=gcrs.WebMercator(), ax=axes[1])
    gplt.pointplot(data[data['label'] == 1], ax=axes[1], alpha=1, s=2, color='k',
                   label='Geolocalization of destroyed buildings')
    gplt.kdeplot(data_preds[data['pred_euroSAT'] == 1], cmap='RdYlGn_r', n_levels=30,
                 shade=True, shade_lowest=False, ax=axes[1], alpha=0.5,
                 label='Predicted density of destroyed buildings')

    axes[0].set_title('Not pretrained model', fontsize=15)
    axes[1].set_title('Pretrained model on euroSAT', fontsize=15)
    plt.legend(bbox_to_anchor=(-0.75, -0.075, 0.5, -0.185), loc='lower left',
               ncol=2, borderaxespad=0., fontsize=15)
    fig.suptitle('Models predictions on Alep', fontsize=20, y=0.95)
    fig.subplots_adjust(wspace=0)

    plt.show()